## Mentiras de trump

## Sumario

Este é um tutorial introdutório sobre captura da Web em Python. Tudo o que é necessário para acompanhar é um entendimento básico da linguagem de programação Python.

No final deste tutorial, você poderá extrair dados de uma página da Web estática usando as bibliotecas ** requests ** e ** Beautiful Soup ** e exportar esses dados para um arquivo de texto estruturado usando os ** pandas * * biblioteca.

## Outline


- O que é web scraping?
- Examinando o artigo do New York Times
    - Examinando o HTML
    - Fato 1: HTML consiste em tags
    - Fato 2: as tags podem ter atributos
    - Fato 3: as tags podem ser aninhadas
- Lendo a página da web em Python
- Analisando o HTML usando Sopa Bonita
    - Coletando todos os registros
    - Extraindo a data
    - Extrair a mentira
    - Extraindo a explicação
    - Extraindo o URL
    - Recapitulação: métodos e atributos da Beautiful Soup
- Construindo o conjunto de dados
    - Aplicando uma estrutura de dados tabular
    

## O que é web scraping?

Em 21 de julho de 2017, o New York Times atualizou um artigo de opinião chamado [Mentiras de Trump] (https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html), detalhando cada público mentira que o presidente disse desde que assumiu o cargo. Porque este é um jornal, a informação foi (obviamente) publicada como um bloco de texto. Este é um ótimo formato para consumo humano, mas não pode ser facilmente entendido por um computador. ** Neste tutorial, vamos extrair as mentiras do presidente do artigo do New York Times e armazená-las em um conjunto de dados não estruturado. **

Esse é um cenário comum: você encontra uma página da Web que contém dados que deseja analisar, mas ela não é apresentada em um formato que você possa baixar e ler facilmente em sua ferramenta de análise de dados favorita. Você pode imaginar copiar e colar manualmente os dados em uma planilha, mas na maioria dos casos, isso consome muito tempo. Uma técnica chamada ** web scraping ** é uma maneira útil de automatizar esse processo.

O que é web scraping? É o processo de extrair informações de uma página da web ** aproveitando os padrões ** no código subjacente da página da web. Vamos começar a procurar por esses padrões!

## Examinando o artigo do New York Times

Aqui está a maneira como o artigo apresentou as informações:

! [Screenshot do artigo] (images / article_1.png)

Ao converter isso em um conjunto de dados **, você pode pensar em cada mentira como um "registro" com quatro campos: **

1. A data da mentira.
2. A própria mentira (como uma citação).
3. A breve explicação do escritor sobre o porquê disso ser uma mentira.
4. A URL de um artigo que substancia a alegação de que era mentira.

É importante ressaltar que esses campos têm formatação diferente, que é consistente em todo o artigo: a data é texto em negrito, a mentira é texto "regular", a explicação é texto cinza itálico e a URL é vinculada ao texto itálico cinza.

** Por que a formatação é importante? ** Porque é muito provável que o código subjacente à página da Web "marque" esses campos de maneira diferente, e podemos aproveitar esse padrão ao copiar a página. Vamos dar uma olhada no código-fonte, conhecido como HTML:

## Examinando o HTML

Para visualizar o código HTML que gera uma página da web, clique com o botão direito do mouse e selecione "Exibir origem da página" no Chrome ou Firefox, "Exibir código-fonte" no Internet Explorer ou "Mostrar código-fonte" no Safari. (Se essa opção não aparecer no Safari, apenas abra as Preferências do Safari, selecione a guia Avançado e marque "Mostrar menu Desenvolver na barra de menus".)

Aqui estão as primeiras linhas que você verá se visualizar a fonte do artigo do New York Times:

! [Captura de tela da fonte] (images / source_1.png)

Vamos localizar a ** primeira mentira ** procurando no HTML pelo texto "iraque":

! [Captura de tela da fonte] (images / source_2.png)

Felizmente, você só precisa entender ** três fatos básicos ** sobre HTML para começar a usar a web scraping!

## Fato 1: HTML consiste em tags

Você pode ver que o HTML contém o texto do artigo, junto com "tags" (especificados usando colchetes angulares) que "marcam" o texto. ("HTML" significa Hyper Text Markup Language.)

Por exemplo, uma tag é `<strong>`, que significa "use formatação em negrito". Existe uma tag `<strong>` antes de "21 de janeiro" e uma tag `</ strong>` após ela. A primeira é uma "tag de abertura" e a segunda é uma "tag de fechamento" (denotada por `/`), que indica ao navegador da web ** por onde começar e parar de aplicar a formatação. ** Em outras palavras, tag informa ao navegador da Web para tornar o texto "21 de janeiro" em negrito. (Não se preocupe com o `& nbsp;` - vamos lidar com isso mais tarde.)

## Fato 2: as tags podem ter atributos

As tags HTML podem ter "atributos", especificados na tag de abertura. Por exemplo, `<span class =" short-desc ">` indica que esta tag `<span>` em particular possui um atributo `class` com um valor de` short-desc`.

Para o propósito de web scraping, ** você não precisa realmente entender ** o significado de `<span>`, `class` ou` short-desc`. Em vez disso, você só precisa reconhecer que as tags podem ter atributos e que elas são especificadas dessa maneira específica.

## Fato 3: as tags podem ser aninhadas

Vamos fingir que meu código HTML disse:

`Olá <strong> <em> alunos da Data School </ em> </ strong>

O texto ** alunos da Data School ** seria em negrito, porque todo esse texto está entre a tag de abertura `<strong>` e a tag de fechamento `</ strong>`. O texto *** Data School *** também estaria em itálico, porque a tag `<em>` significa "use itálico". O texto "Hello" não seria em negrito ou itálico, porque não está dentro das tags `<strong>` ou `<em>`. Assim, apareceria da seguinte forma:

Olá *** alunos da Data School * **

O ponto central a ser seguido neste exemplo é que as tags ** "marcam" o texto de onde quer que elas abram para onde quer que sejam fechadas, independentemente de estarem aninhadas em outras tags.

Consegui? Agora você sabe o suficiente sobre o HTML para começar a usar a web!

## Lendo a página da web em Python

A primeira coisa que precisamos fazer é ler o HTML deste artigo no Python, o que faremos usando a biblioteca [requests] (http://docs.python-requests.org/en/master/). (Se você não tiver, você pode `pip install requests` da linha de comando.)

In [0]:
import requests
r = requests.get('https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html')

O código acima busca nossa página web a partir do URL e armazena o resultado em um objeto "resposta" chamado `r`. Esse objeto de resposta tem um atributo `text`, que contém o mesmo código HTML que vimos ao visualizar a fonte em nosso navegador:

In [0]:
# print the first 500 characters of the HTML
print(r.text[0:500])

## Analisando o HTML usando beautifulsoup

Vamos analisar o HTML usando a biblioteca [Beautiful Soup 4] (https://www.crummy.com/software/BeautifulSoup/bs4/doc/), que é uma biblioteca popular em Python para a criação de páginas da web. (Se você não tiver, você pode `pip instalar beautifulsoup4` a partir da linha de comando.)

In [0]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')

O código acima analisa o HTML (armazenado em `r.text`) em um objeto especial chamado` soup` que a biblioteca Beautiful Soup entende. Em outras palavras, o Beautiful Soup está ** lendo o HTML e dando sentido a sua estrutura. **

(Observe que `html.parser` é o analisador incluído na biblioteca padrão do Python, embora outros analisadores possam ser usados ​​pelo Beautiful Soup. Consulte [diferenças entre analisadores] (https://www.crummy.com/software/BeautifulSoup/bs4 / doc / # difference-between-parsers) para aprender mais.)

## Coletando todos os registros

O código Python acima é o código padrão que uso em todos os projetos de varredura da web. Agora, vamos começar ** aproveitando os padrões que observamos na formatação do artigo ** para criar nosso conjunto de dados!

Vamos dar uma outra olhada no artigo e compará-lo com o HTML:

! [Screenshot do artigo] (images / article_1.png)

! [Captura de tela da fonte] (images / source_3.png)

Você deve ter notado que cada registro tem o seguinte formato:

`<span class =" short-desc "> <strong> DATA </ strong> MENTIRA <span class =" short-truth "> <a href="URL"> EXPLICAÇÃO </a> </ span> </ span > `

Há uma tag `<span>` externa e, em seguida, aninhada dentro dela é uma tag `<strong>` e outra tag <span>, que contém uma tag `<a>`. Todas essas tags afetam a formatação do texto. E como o New York Times quer que cada registro apareça de maneira consistente em seu navegador, sabemos que ** cada registro será marcado de maneira consistente no HTML. ** Esse é o padrão que nos permite construir nossa conjunto de dados!

Vamos pedir para o Beautiful Soup ** encontrar todos os registros: **

In [0]:
results = soup.find_all('span', attrs={'class':'short-desc'})

Este código procura o objeto `soup` por todas as tags` <span> `com o atributo` class = "short-desc" `. Ele retorna um objeto especial Beautiful Soup (chamado de "ResultSet") contendo os resultados da pesquisa.

`results` atua como uma lista ** do Python **, para que possamos verificar sua duração:

In [0]:
len(results)

Existem 180 resultados, o que parece razoável dada a duração do artigo. (Se esse número não parecer razoável, examinaríamos o HTML ainda mais para determinar se nossas suposições sobre os padrões no HTML estavam incorretas.)

Também podemos dividir o objeto como uma lista, para examinar os ** primeiros três resultados: **

In [0]:
results[0:3]

ambém verificaremos se o ** último resultado ** deste objeto corresponde ao último registro no artigo:

! [Screenshot do artigo] (images / article_2.png)

In [0]:
results[-1]

Parece bom!

Nós agora coletamos todos os 116 registros, mas ainda precisamos separar cada registro em seus quatro componentes (data, mentira, explicação e URL) para dar ao conjunto de dados alguma estrutura.

## Extraindo a data

O Web scraping é geralmente um processo interativo, no qual você experimenta seu código até que ele funcione exatamente como deseja. Para simplificar a experimentação, começaremos trabalhando apenas com o ** primeiro registro ** no objeto `results` e, mais tarde, modificaremos nosso código para usar um loop:

In [0]:
first_result = results[0]
first_result


Embora `first_result` possa parecer uma string Python, você notará que não há marcas de aspas em torno dela. Em vez disso, é outro objeto especial Beautiful Soup (chamado de "Tag") que possui métodos e atributos específicos.

Para localizar a data, podemos usar seu método `find ()` para ** encontrar uma única tag ** que corresponda a um padrão específico, em contraste com o método `find_all ()` que usamos acima para ** encontrar todas tags ** que correspondem a um padrão:

In [0]:
first_result.find('strong')

Este código pesquisa `first_result` pela primeira instância de uma tag` <strong> `e retorna novamente um objeto" Tag "de Beautiful Soup (não uma string).

Como queremos ** extrair o texto entre as tags de abertura e fechamento **, podemos acessar seu atributo `text`, que de fato retorna uma string normal do Python:

In [0]:
first_result.find('strong').text

O que é `\ xa0`? Na verdade, você não precisa saber disso, mas é chamado de "seqüência de escape" que representa o caractere "& nbsp;" que vimos anteriormente no código-fonte HTML.

No entanto, você precisa saber que ** uma seqüência de escape representa um único caractere ** dentro de uma string. Vamos dividi-lo a partir do final da string:

In [0]:
first_result.find('strong').text[0:-1]

Por fim, adicionaremos o ano, pois não queremos que nosso conjunto de dados inclua datas ambíguas:

In [0]:
first_result.find('strong').text[0:-1] + ', 2017'


## Extraindo a mentira

Vamos dar uma outra olhada no `first_result`:

In [0]:
first_result

osso objetivo é extrair as duas frases sobre o Iraque. Infelizmente, não há um par de tags de abertura e fechamento que começa **immediately before the lie** and ends **immediately after the lie**. Portanto, vamos ter que usar uma técnica diferente:

In [0]:
first_result.contents


O tag "first_result`" possui um atributo `contents`, que retorna uma lista do Python contendo seus" filhos ". O que são crianças? São as tags e strings que estão aninhadas em uma tag.

Podemos dividir essa lista para extrair o segundo elemento:

In [0]:
first_result.contents[1]


Por fim, dividiremos as aspas curvas e o espaço extra no final:

In [0]:
first_result.contents[1][1:-2]

# Extraindo a explicação

Com base no que você já viu, você pode ter descoberto que temos pelo menos ** duas opções ** de como extraímos o terceiro componente do registro, que é a explicação do autor da razão pela qual a declaração do Presidente foi uma mentira.

A ** primeira opção ** é dividir o atributo `contents`, como fizemos ao extrair a mentira:

In [0]:
first_result.contents[2]

A ** segunda opção ** é pesquisar a tag ao redor, como fizemos ao extrair a data:

In [0]:
first_result.find('a')

De qualquer forma, podemos acessar o atributo `text` e depois cortar os parênteses de abertura e fechamento:

In [0]:
first_result.find('a').text[1:-1]

## Extraindo o URL

Finalmente, queremos extrair a URL do artigo que substancia a afirmação do escritor de que o presidente estava mentindo.

Vamos examinar a tag `<a>` dentro de `first_result`:

In [0]:
first_result.find('a')

Até agora, neste tutorial, extraímos o texto ** entre as tags **. Nesse caso, o texto que queremos extrair está localizado ** dentro da própria tag **. Especificamente, queremos acessar o valor do atributo `href` dentro da tag` <a> `.

A Beautiful Soup trata os atributos da tag e seus valores como ** pares de valores-chave em um dicionário: ** você coloca o nome do atributo entre colchetes (como uma chave de dicionário) e recupera o valor do atributo:

In [0]:
first_result.find('a')['href']

## Recap: métodos e atributos da Beautiful Soup

Antes de concluirmos a construção do conjunto de dados, quero resumir algumas maneiras pelas quais você pode interagir com os objetos Beautiful Soup.

Você pode aplicar esses ** dois métodos ** ao objeto inicial `soup` ou a um objeto Tag (como` first_result`):

- `find ()`: procura pela primeira tag correspondente e retorna um objeto Tag
- `find_all ()`: procura por todas as tags correspondentes e retorna um objeto ResultSet (que você pode tratar como uma lista de Tags)

Você pode extrair informações de um objeto Tag (como `first_result`) usando esses ** dois atributos: **

- `text`: extrai o texto de um Tag e retorna uma string
- `contents`: extrai os filhos de um Tag e retorna uma lista de Tags e strings

É importante saber se você está interagindo com um Tag, um ResultSet, uma lista ou uma string, porque isso afeta quais métodos e atributos você pode acessar.

E, claro, há muitos outros métodos e atributos disponíveis para você, descritos na [documentação da Beautiful Soup] (https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

## Criando o conjunto de dados

Agora que descobrimos como extrair os quatro componentes de `first_result`, podemos ** criar um loop para repetir este processo ** em todos os 116` results`. Nós vamos armazenar a saída em uma ** lista de tuplas ** chamada `registros`:

In [0]:
records = []
for result in results:
    date = result.find('strong').text[0:-1] + ', 2017'
    lie = result.contents[1][1:-2]
    explanation = result.find('a').text[1:-1]
    url = result.find('a')['href']
    records.append((date, lie, explanation, url))

Como havia 116 `resultados`, deveríamos ter 116` registros`:

In [0]:
len(records)

Vamos fazer uma verificação rápida dos três primeiros registros:

In [0]:
records[0:3]

Boa sorte

## Aplicando uma estrutura de dados tabular

O último grande passo neste processo é aplicar uma estrutura de dados tabular à nossa estrutura existente (que é uma lista de tuplas). Faremos isso usando a biblioteca [pandas] (http://pandas.pydata.org/), uma biblioteca Python incrivelmente popular para análise e manipulação de dados. (Se você não tem, aqui estão as [instruções de instalação] (http://pandas.pydata.org/pandas-docs/stable/install.html).)

A estrutura de dados primária em pandas é o "DataFrame", que é adequado para dados tabulares com colunas de diferentes tipos, ** semelhante a uma planilha do Excel ou tabela SQL. ** Podemos converter nossa lista de tuplas em um DataFrame passando-o ao construtor DataFrame e especificando os nomes de coluna desejados:

In [0]:
import pandas as pd
df = pd.DataFrame(records, columns=['date', 'lie', 'explanation', 'url'])

O DataFrame inclui um método `head ()`, que permite examinar a parte superior do DataFrame:

In [0]:
df.head()

The numbers on the left side of the DataFrame are known as the "index", which act as identifiers for the rows. Because we didn't specify an index, it was automatically assigned as the integers 0 to 115.

We can examine the bottom of the DataFrame using the `tail()` method:

In [0]:
df.tail()

Did you notice that "January" is abbreviated, while "July" is not? It's best to format your data consistently, and so we're going to convert the date column to pandas' special "datetime" format:

In [0]:
df['date'] = pd.to_datetime(df['date'])

The code above converts the "date" column to datetime format, and then overwrites the existing "date" column. (Notice that we did not have to tell pandas that the column was originally in "MONTH DAY, YEAR" format - **pandas just figured it out!**)

Let's take a look at the results:

In [0]:
df.head()

In [0]:
df.tail()

Not only is the date column now consistently formatted, but pandas also provides a wealth of [date-related functionality](https://pandas.pydata.org/pandas-docs/stable/timeseries.html) because it's in datetime format.